In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
    
    def forward(self, src):
        src2 = self.self_attn(src, src, src, attn_mask=src_mask)[0]
        src = src + self.dropout(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout(src2)
        src = self.norm2(src)
        return src

class TransformerEncoder(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, num_layers, dropout):
        super(TransformerEncoder, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers)
    
    def forward(self, src, src_mask=None):
        src = self.encoder(src, src_mask=src_mask)
        return src

class BearingFaultModel(nn.Module):
    # def __init__(self, input_dim, hidden_dim, output_dim, num_layers, nhead, dim_feedforward, dropout):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, nhead, dim_feedforward, dropout):
        super(BearingFaultModel, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        # self.transformer_encoder = TransformerEncoder(hidden_dim, nhead, dim_feedforward, num_layers, dropout)
        encoder_layer = nn.TransformerEncoderLayer(hidden_dim, nhead, dim_feedforward, dropout)
        # self.transformer_encoder = TransformerEncoder(hidden_dim, nhead, dim_feedforward, num_layers, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, src):
        src = self.embedding(src)
        src = src.permute(1, 0, 2)
        src = self.transformer_encoder(src)
        src = src.permute(1, 0, 2)
        src = torch.mean(src, dim=1)  # 平均池化
        src = self.fc(src)
        return src

# 定义模型参数
input_dim = 1024
hidden_dim = 256
output_dim = 10
num_layers = 2
nhead = 4
dim_feedforward = 512
dropout = 0.1

# 创建模型实例
model = BearingFaultModel(input_dim, hidden_dim, output_dim, num_layers, nhead, dim_feedforward, dropout)

# 输出模型结构
print(model)

BearingFaultModel(
  (embedding): Linear(in_features=1024, out_features=256, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linea

In [12]:
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim

In [13]:
# Task 1
task1_data = np.load('./cwru1_X_train.npy')
task1_labels = np.load('./cwru2_y_train.npy')

# Task 2
task2_data = np.load('./cwru2_X_train.npy')
task2_labels = np.load('./cwru2_y_train.npy')

In [14]:
# Task 1
task1_data = torch.from_numpy(task1_data)
task1_labels = torch.from_numpy(task1_labels)

# Task 2
task2_data = torch.from_numpy(task2_data)
task2_labels = torch.from_numpy(task2_labels)

In [7]:
task1_data[0].shape

torch.Size([1024, 2])

In [15]:
# data transpose
task1_data = task1_data.transpose(1, 2)
task2_data = task2_data.transpose(1, 2)

In [16]:
from sklearn.metrics import accuracy_score

# 数据准备
# 假设您已经将数据准备成训练集和测试集的形式，分别为train_data, train_labels, test_data, test_labels

# 转换为Tensor格式
# train_data = torch.Tensor(task1_data)
# train_labels = torch.LongTensor(task1_labels)
# test_data = torch.Tensor(task2_data)
# test_labels = torch.LongTensor(task2_labels)
train_data = task1_data
train_labels = task1_labels
test_data = task2_data
test_labels = task2_labels

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 创建模型实例
# model = BearingFaultModel(input_dim, hidden_dim, output_dim, num_layers, nhead, dim_feedforward, dropout)
model = BearingFaultModel(input_dim, hidden_dim, output_dim, num_layers, nhead, dim_feedforward, dropout).to(device)
model.double()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 模型训练
num_epochs = 10
batch_size = 32

for epoch in range(num_epochs):
    model.train()
    for i in range(0, len(train_data), batch_size):
        batch_data = train_data[i:i+batch_size].to(device, dtype=torch.double)
        batch_labels = train_labels[i:i+batch_size].to(device, dtype=torch.long)
        
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
    
    # 模型评估
    model.eval()
    with torch.no_grad():
        outputs = model(test_data)
        
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(test_labels, predicted)
        print(f"Epoch {epoch+1} - Accuracy: {accuracy}")

Epoch 1 - Accuracy: 0.30257142857142855
Epoch 2 - Accuracy: 0.32821428571428574
Epoch 3 - Accuracy: 0.35414285714285715
Epoch 4 - Accuracy: 0.3637142857142857
Epoch 5 - Accuracy: 0.3226428571428571
Epoch 6 - Accuracy: 0.3455
Epoch 7 - Accuracy: 0.3999285714285714
Epoch 8 - Accuracy: 0.3425714285714286
Epoch 9 - Accuracy: 0.3777142857142857
Epoch 10 - Accuracy: 0.3653571428571429
